# Segmenting and Clustering Neighborhoods in Toronto

## Introduction

In this assignment, we will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so we need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. We will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format.

Once the data is in a structured format, we can replicate the analysis to explore and cluster the neighborhoods in the city of Toronto.

### Import libraries

In [27]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from opencage.geocoder import OpenCageGeocode
from time import sleep
from IPython.display import display
import folium

### Downloading and parsing the data to work with

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [4]:
raw = []
#going through tr objects
for tr in soup.find('tbody'):
    #going through td objects
    for td in tr:
        #extracting content of each td object
        for element in td:
            #checking if the object is not empty line
            if (element != '\n'):
                #checking if object has another tag inside, if so - extract
                if (element.find('<')!=-1):
                    s0up = BeautifulSoup(str(element), 'lxml')
                    element = s0up.find('a').text
                #checking for newlines on the end of elements, if so - cut
                if "\n" in element:
                    element = element.rstrip('\n')
                #appending the extracted element to the list    
                raw.append(str(element))

### Cleaning up from empty Boroughs

In [5]:
for i in range(0, len(raw)):
    if(raw[i]==raw[i-1]):
        raw[i-2] = ""
        raw[i-1] = ""
        raw[i] = ""      
clean = list(filter(None, raw))

### Setting "Not assigned" Neighbourhood value equal to the Borough

In [7]:
for i in range(0, len(clean)):      
    if(clean[i]=='Not assigned'):
        clean[i] = clean[i-1]

### Splitting the list into dataframe by categories (Postcode, Borough, Neighbourhood)

In [9]:
#splitting list by rows
split = np.array_split(clean,len(clean)/3)
#write to dataframe with using data from first row elements as headers
df = pd.DataFrame(split[1:], columns=split[0])
#combining Neighbourhoods to corresponding postcodes and dropping duplicates
df['Neighbourhood'] = df.groupby(['Postcode','Borough'])['Neighbourhood'].transform(lambda x: ', '.join(x)) 
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

### Checking the shape of cleaned dataset and printing first columns to see formating

In [19]:
display(df.shape)
display(df.head(10))

(103, 3)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


### Initializing the geocordinates plugin

In [22]:
key = '96477f6f7c8e46f2b79da229da28a656'
geocoder = OpenCageGeocode(key)
lat = []
lng = []

### Getting coordinates for all postal codes through the loop

In [23]:
for item in df['Postcode']:
    res = geocoder.geocode('{}, Toronto, Ontario'.format(item))
    #appending coordinates to coresponding lists
    lat.append(res[0]['geometry']['lat'])
    lng.append(res[0]['geometry']['lng'])
    #sleep for 1.1 second - delay between requests
    sleep(1.1)
print ('data recieving finished')

data recieving finished


### Adding recieved coordinates to our table

In [24]:
df['Latitude'] = lat
df['Longitude'] = lng

### Checking the shape of cleaned dataset and printing first columns to see formating

In [42]:
display(df.shape)
display(df.head(10))

(103, 5)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.754500,-79.330000
1,M4A,North York,Victoria Village,43.727600,-79.314800
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.655500,-79.362600
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.722300,-79.450400
4,M7A,Queen's Park,Queen's Park,43.653963,-79.387207
5,M9A,Etobicoke,Islington Avenue,43.666200,-79.528200
6,M1B,Scarborough,"Rouge, Malvern",43.653963,-79.387207
7,M3B,North York,Don Mills North,43.745000,-79.359000
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706300,-79.309400
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657200,-79.378300


### Visualizing the regions of Torronto and it's Neighbourhoods by postcodes

In [45]:
#getting geoposition of Torronto
cord = geocoder.geocode('Toronto, Ontario')

#create map of Torronto using latitude and longitude values
map_torronto = folium.Map(location=[cord[0]['geometry']['lat'], cord[0]['geometry']['lng']], zoom_start=11)

#add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='darkgreen',
        fill=True,
        fill_color='#ffdf5b',
        fill_opacity=0.7,
        parse_html=False).add_to(map_torronto)  

#printing the map
map_torronto